<a href="http://laf-fabric.readthedocs.org/en/latest/" target="_blank"><img align="left" src="images/laf-fabric-xsmall.png"/></a>
<a href="http://emdros.org" target="_blank"><img align="left" src="files/images/Emdros-xsmall.png"/></a>
<a href="http://www.persistent-identifier.nl/?identifier=urn%3Anbn%3Anl%3Aui%3A13-048i-71" target="_blank"><img align="left"src="images/etcbc4easy-small.png"/></a>
<a href="http://www.godgeleerdheid.vu.nl/etcbc" target="_blank"><img align="right" src="images/VU-ETCBC-xsmall.png"/></a>

# Cardinals and verbs

Which verbs show up in the context of numbers, and which syntactic roles do numbers assume by virtue of their link to those verbs and which meanings do they get?

This is a question coming from Oliver Glanz at Andrew University.

In [1]:
import sys
import collections
import subprocess

from lxml import etree

import laf
from laf.fabric import LafFabric
from etcbc.preprocess import prepare
from etcbc.mql import MQL
fabric = LafFabric()

  0.00s This is LAF-Fabric 4.4.6
API reference: http://laf-fabric.readthedocs.org/en/latest/texts/API-reference.html
Feature doc: http://shebanq-doc.readthedocs.org/en/latest/texts/welcome.html



In [2]:
API = fabric.load('etcbc4', '--', 'mql', {
    "xmlids": {"node": False, "edge": False},
    "features": ('''
        oid otype monads
        lex_utf8
        sp typ vs function
        book chapter verse label
    ''','''
        functional_parent
    '''),
    "prepare": prepare,
    "primary": True,
}, verbose='DETAIL')
exec(fabric.localnames.format(var='fabric'))
Q = MQL(API)

  0.00s LOADING API: please wait ... 
  0.00s DETAIL: COMPILING m: UP TO DATE
  0.10s INFO: USING DATA COMPILED AT: 2014-07-23T09-31-37
  0.10s DETAIL: COMPILING a: UP TO DATE
  0.11s DETAIL: load main: P.node_anchor
  0.80s DETAIL: load main: P.node_anchor_items
  1.04s DETAIL: load main: G.node_anchor_min
  1.10s DETAIL: load main: G.node_anchor_max
  1.14s DETAIL: load main: P.node_events
  1.23s DETAIL: load main: P.node_events_items
  1.51s DETAIL: load main: P.node_events_k
  1.60s DETAIL: load main: P.node_events_n
  1.74s DETAIL: load main: G.node_sort
  1.79s DETAIL: load main: G.node_sort_inv
  2.18s DETAIL: load main: G.edges_from
  2.25s DETAIL: load main: G.edges_to
  2.32s DETAIL: load main: P.primary_data
  2.37s DETAIL: load main: F.etcbc4_db_monads [node] 
  3.11s DETAIL: load main: F.etcbc4_db_oid [node] 
  3.81s DETAIL: load main: F.etcbc4_db_otype [node] 
  4.45s DETAIL: load main: F.etcbc4_ft_function [node] 
  4.57s DETAIL: load main: F.etcbc4_ft_lex_utf8 [node] 


# The MQL query

The query that selects the relevant fragments is on SHEBANQ:
[Oliver Glanz: Lev 26 function of numbers #01](http://shebanq.ancient-data.org/hebrew/query?id=544)

In [3]:
card_query = '''
select all objects where
[book [chapter [verse
[clause
	[phrase
		[word sp = verb]
	]
	..
	[phrase
		[word FIRST AND LAST ls  = card AND prs = "absent"]
	]
] 
OR
[clause
	[phrase
		[word FIRST AND LAST ls  = card AND prs = "absent"]
	]
	..
	[phrase
		[word sp = verb]
	]
]
]]]
'''

# Run the query

The following line of code invokes your local MQL processor (from your local EMDROS installation), and delivers the results as a compact sheaf.

In [4]:
sheaf = Q.mql(card_query)

# Processing the results

Now we process all results. Every result is a structure in the same shape as our query, filled with nodes that correspond to the book, chapter, verse, clause, phrase etc. of that result.

We iterate through all these objects, identify the nodes in it, and look up desired features of these nodes.

It is a bit daunting, this data structure, but it is easy to get it.
We just print out the first result of the sheaf and copy the result to our program code, and them replace the numbers by variables.


In [9]:
for x in sheaf.results():
    print(x)
    break

((1372281, ((1372323, ((1418025, ((426940, ((605825, ((1912,),)), (605826, ((1914,),)))),)),)),)),)


For each result, we write out a line of feature information.

In [5]:
tsvfile = outfile('card.csv')
nresults = 0
tsvfile.write('book\tchapter\tverse\torder\tclause typ\tverbal stem\tverb\tcard function\tcard\tphrase verb text\tphrase card text\tclause text\n')
for ((book, ((chapter, ((verse, ((clause, ((phrase1, ((word1,),)), (phrase2, ((word2,),)))),)),)),)),) in sheaf.results():
    nresults += 1
    verb_first = F.sp.v(word1) == 'verb'
    word_verb = word1 if verb_first else word2
    word_card = word2 if verb_first else word1
    phrase_verb = phrase1 if verb_first else phrase2
    phrase_card = phrase2 if verb_first else phrase1
    clause_text = ' ~ '.join(x[1] for x in P.data(clause))
    phrase_verb_text = ' ~ '.join(x[1] for x in P.data(phrase_verb))
    phrase_card_text = ' ~ '.join(x[1] for x in P.data(phrase_card))
    
    tsvfile.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(
        F.book.v(book), 
        F.chapter.v(chapter), 
        F.verse.v(verse),
        '>>' if verb_first else '<<',
        F.typ.v(clause),
        F.vs.v(word_verb),
        F.lex_utf8.v(word_verb),
        F.function.v(phrase_card),
        F.lex_utf8.v(word_card),
        phrase_verb_text,
        phrase_card_text,
        clause_text,
    ))
tsvfile.close()
print('There are {} results'.format(nresults))

There are 116 results


In [6]:
!head -n 10 {my_file('card.csv')}

book	chapter	verse	order	clause typ	verbal stem	verb	card function	card	phrase verb text	phrase card text	clause text
Genesis	4	15	>>	XYqt	qal	הרג[	Adju	שׁבע/	כָּל ~ הֹרֵ֣ג ~ קַ֔יִן	שִׁבְעָתַ֖יִם	כָּל ~ הֹרֵ֣ג ~ קַ֔יִן ~ שִׁבְעָתַ֖יִם ~ יֻקָּ֑ם
Genesis	4	15	<<	XYqt	hof	נקם[	Adju	שׁבע/	יֻקָּ֑ם	שִׁבְעָתַ֖יִם	כָּל ~ הֹרֵ֣ג ~ קַ֔יִן ~ שִׁבְעָתַ֖יִם ~ יֻקָּ֑ם
Genesis	4	24	<<	xYqX	hof	נקם[	Adju	שׁבע/	יֻקַּם	שִׁבְעָתַ֖יִם	כִּ֥י ~ שִׁבְעָתַ֖יִם ~ יֻקַּם ~ קָ֑יִן
Genesis	18	28	>>	xYqX	qal	חסר[	Subj	חמשׁ/	יַחְסְר֞וּן	חֲמִשָּׁ֔ה	א֠וּלַי ~ יַחְסְר֞וּן ~ חֲמִשִּׁ֤ים ~ הַצַּדִּיקִם֙ ~ חֲמִשָּׁ֔ה
Genesis	18	29	>>	xYqX	nif	מצא[	Subj	ארבע/	יִמָּצְא֥וּן	אַרְבָּעִ֑ים	אוּלַ֛י ~ יִמָּצְא֥וּן ~ שָׁ֖ם ~ אַרְבָּעִ֑ים
Genesis	18	30	>>	xYqX	nif	מצא[	Subj	שׁלשׁ/	יִמָּצְא֥וּן	שְׁלֹשִׁ֑ים	אוּלַ֛י ~ יִמָּצְא֥וּן ~ שָׁ֖ם ~ שְׁלֹשִׁ֑ים
Genesis	18	30	>>	xYq0	qal	מצא[	Objc	שׁלשׁ/	אֶמְצָ֥א	שְׁלֹשִֽׁים	אִם ~ אֶמְצָ֥א ~ שָׁ֖ם ~ שְׁלֹשִֽׁים
Genesis	18	31	>>	xYqX	nif	מצא[	Subj	עשׂרים/	יִמָּצְא֥וּן	עֶשְׂרִ֑ים	אוּלַ֛י ~ יִמָּ

Download the result files from my SURFdrive: [tab separated file](https://surfdrive.surf.nl/files/public.php?service=files&t=b9706323fd50dc7e8a53f2c62f16d16d) and a formatted [openoffice spreadsheet](https://surfdrive.surf.nl/files/public.php?service=files&t=8a42b437d2447e44041b8a3add073104)